In [ ]:
#import libraries used into
from datetime import datetime, timedelta
import pandas as pd, numpy as np, os

In [ ]:
##First, the vars was created
fileam = "OT " + datetime.now().strftime('%d.%m.%Y') + ".XLSX"
print("fileam: ", fileam)
if datetime.today().strftime('%A') == "Monday":
    fileyest = "OT " + (datetime.today() - timedelta(days = 3)).strftime('%d.%m.%Y' + ".XLSX")
    print("fileyest: ", fileyest)
else:
    fileyest = "OT " + (datetime.today() - timedelta(days = 1)).strftime('%d.%m.%Y' + ".XLSX")
    print("fileyest: ", fileyest)
filepm = "OT " + datetime.now().strftime('%d.%m.%Y') + " 5 P.M..XLSX"
print("filepm: ", filepm)
file3pm = "OT " + datetime.now().strftime('%d.%m.%Y') + " 3 P.M..XLSX"
print("file3pm: ", file3pm)
main_path = "C:\\Users\\mruiz\\Documents\\DB OT\\" 
current_path = "C:\\Users\\mruiz\\Documents\\DB OT\\CurrentDocuments"
current_month = datetime.now().strftime('%B')
current_path_ = os.listdir(current_path)
main_path_ = os.listdir(main_path)
data_yesterday_excel = {}
data_am_excel = {}
data_3pm_excel = {}
data_pm_excel = {}
repro_nuevas = 1

PRUEBAS

Fin de las pruebas

In [ ]:
def get_data(entry_data):
    file = main_path + entry_data
    filexcel = pd.read_excel(file,sheet_name="Sheet1")
    print("Los datos del archivo ", entry_data, " fueron procesados")
    return filexcel

In [ ]:
while True:
    try:
        data_yesterday_excel = get_data(fileyest)
        data_am_excel = get_data(fileam)
        break
    except FileNotFoundError:
        print("Hay al menos un archivo que no fue leído: ")
        print("Archivo Ayer : ", type(data_yesterday_excel))
        print("Archivo hoy por la mañana: ", type(data_am_excel))
        break

In [ ]:
if type(data_am_excel) == type(data_yesterday_excel):
    programa = data_am_excel
    print("Los datos de la mañana fueron seleccionados")
else :
    print("Error")

In [ ]:
data2 =current_path + "\\Master_OT.xlsx"
nombres = pd.read_excel(data2, sheet_name="Names", header=0)

In [ ]:
data3 = current_path + "\\Politicas.XLSX"
politicas = pd.read_excel(data3, sheet_name="POLITICAS CONTRATO", header=0)

In [ ]:
data4 = current_path + "\\Materiales_Locales.xlsx"
mlocal = pd.read_excel(data4, sheet_name="ENTREGADO", header=0)

In [ ]:
data5 = current_path + "\\Materiales_Foraneos.xlsx"
mforaneo = pd.read_excel(data5, sheet_name="MATRIZ FINAL", header=1)

In [ ]:
data6 = current_path + "\\Notificadas_Mensual.xlsx"
reportes = pd.read_excel(data6, sheet_name="CONTROL_REMASTERED", header=0)

In [ ]:
data7 = current_path + "\\Ubicaciones_Equipos.xlsx"
ubicaciones = pd.read_excel(data7, sheet_name="Sheet1", header=0)

In [ ]:
data8 = current_path + "\\Reprogramaciones.xlsx"
reprogramaciones_ = pd.read_excel(data8, sheet_name="DB", header=0)

In [ ]:
reprogramaciones = data_yesterday_excel.copy()

In [ ]:
reprogramaciones.rename(columns={'Inicio programado':'Fecha_reprogramada'}, inplace=True)

In [ ]:
programa['Material'] = np.where((programa['Clase de orden'] == 'ZMCP') &  (programa['Texto breve'].str.contains('Emer|Revi', case=False)), 'NO',
np.where(programa['Clase de orden'] == 'ZPM2','SI',
np.where(programa['Clase de orden'] == 'ZMCP','SI',
np.where(programa['Clase de orden'] == 'ZMCI', 'SI',
np.where((programa['Clase de orden'] == 'ZPM1') &  (programa['Texto breve'].str.contains('CAFA|CABAT|ACUM', case=False)), 'SI','NO')))))

In [ ]:
#print('Las columnas de frame 1 son', programa.info())
#print('Las columnas de frame 2 son', nombres.columns)
#print('Las columnas de frame 3 son', politicas.columns)
#print('Las columnas de frame 4 son', mlocal.columns)
#print('Las columnas de frame 5 son', mforaneo.columns)
#print('Las columnas de reprogramaciones son ',reprogramaciones)
#print('Las columnas de frame 6 son', reportes.columns)
#print('Las columnas de frame 7 son', ubicaciones.columns)
#print('Las columnas de frame main son', result_df.columns)

In [ ]:
reportes['Inicio programado'] = reportes['Inicio programado'].str.replace("-","/")
reportes['Inicio programado'] = pd.to_datetime(reportes['Inicio programado'], format = "%d/%m/%Y", exact=False)
print(reportes['Inicio programado'].dtype)

In [ ]:
result_df = pd.merge(programa,reportes,on=['Inicio programado','Orden','Pto.tbjo.responsable','Cliente','Denominación de objeto técnico',
'Número de serie','Equipo','Clase de orden'],how="outer")

In [ ]:
result_df = pd.merge(left = result_df, right = nombres[['TecID','Short name','Estatus','City']],left_on="Pto.tbjo.responsable",right_on="TecID",how="left")

In [ ]:
result_df = pd.merge(result_df,mlocal[['Orden', 'Inicio programado2', 'Fecha de salida',
       'Comentario', 'Material Separado']], on=['Orden'],how= "left")

In [ ]:
result_df = pd.merge(result_df,mforaneo[['Orden','UBICACIÓN', 'ESTATUS COMPRA', 'PROVEEDOR']],on='Orden',how='left')

In [ ]:
result_df = pd.merge(left = result_df,right = reprogramaciones[['Orden','Fecha_reprogramada']],on='Orden',how='left')

In [ ]:
result_df = pd.merge(left=result_df,right=ubicaciones[['Equipo','Dirección']],how='left',on='Equipo')

In [ ]:
politicas_columns = ['Pto.tbjo.responsable', 'Status del sistema', 'Denominación de objeto técnico', 'Número de serie',
'Denominación de objeto técnico', 'Status del sistema', 'Número de serie', 'Pto.tbjo.responsable','OC','Equipo']

In [ ]:
result_df = pd.merge(result_df,politicas,on = politicas_columns,how="left")

In [ ]:
listr = ['Equipo','Pto.tbjo.responsable', 'Status del sistema','Denominación de objeto técnico', 'Número de serie',
'OC','Cliente actual', 'Denominación de la ubicación técnica', 'gestora archivo anahi ', 'reportes archivo anahi ',
'Tipo Equipo', 'Frecuencia', 'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.', 'Horario', 'Curso', 'DC-3',
'Examenes Covid, antidoping, etc', 'Tiempo valido de prueba', 'Tecnicos para servicio', 'Requerimientos adicionales', 
'Correo solicitud acceso', 'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales', 'FRECUENCIA DE OC', 'PREVENTIVOS',
'CORRECTIVOS']

In [ ]:
result_df = pd.merge(result_df,politicas,on=[listr],how="left")

In [ ]:
print(reportes.columns.values)

In [ ]:
result_df = result_df.assign(Pendiente_Recoger=np.where(result_df['Inicio programado2'].isnull(),result_df['ESTATUS COMPRA'],result_df['Inicio programado2']))

In [ ]:
result_df.assign(reprogramacion=0)

In [ ]:
reprogramaciones = result_df.copy()

In [ ]:
result_df['reprogramacion'] = np.where(result_df['Inicio programado']==reprogramaciones['Fecha_reprogramada'], "Sin_Reprogramar","Reprogramada")
result_df['reprogramacion'] = np.where(result_df['Fecha_reprogramada'].isnull(),"O.T. Agregada",result_df['reprogramacion'])

In [ ]:
print(result_df['reprogramacion'].unique())

In [ ]:
result_df.assign(Politica=0)

In [ ]:
print(list(result_df.columns.values))

In [ ]:
result_df['Politica'] = np.where(result_df['Requiere Acceso'].isnull(),"NO","SI")

In [ ]:
print(result_df.shape[0])
print(programa.shape[0])

In [ ]:
reprogramaciones = result_df[result_df['reprogramacion'].isin(['Reprogramada'])]
if datetime.today().strftime('%Y-%m-%d') == reprogramaciones_['Dia_reprogramado'].loc[0]:
       repro_nuevas = 0
       print("Sin Reprogramaciones nuevas")
       reprogramaciones = reprogramaciones_
else:
       reprogramaciones = reprogramaciones.assign(Dia_reprogramado = datetime.today().strftime('%Y-%m-%d'))
       reprogramaciones = pd.concat([reprogramaciones,reprogramaciones_])
       print(reprogramaciones.shape[0])

In [ ]:
listr = np.where(reportes.columns.str.contains('ESTATUS'), reportes.columns,"")
i = 0
eliminate = []
for element in listr:
    if element == "" : 
        eliminate.append(i)
    i = i+1
listr = np.delete(listr,eliminate)
estatus_actual = listr[-1]
listr = np.delete(listr,np.where(listr == estatus_actual))

In [ ]:
listr_ = ['COMENTARIO ','Tipo Cliente','MES ','Mes2','Antiguedad','Gestora','Gpo Cliente','FORMATO ESPECIAL',
'SEMANA ANTERIORES ','FACTURA ','SOLO AMARILLO ','REPORTE INCOMPLETO ','LEVANTAMIENTO ','CONTRA RECIBO ',
'Gestora','preventivos','correctivos','comentarios internos adiccionales','Gestora.1','TecID']
mylist = np.concatenate((listr,listr_),axis=None)
#reportes = reportes.drop(mylist,axis=1)

In [ ]:
reprogramaciones = reprogramaciones[['Orden', 'Inicio programado','Plan mant.preventivo', 'Pto.tbjo.responsable',
       'Status del sistema', 'Texto breve', 'Cliente', 'Fecha inicio real',
       'Fecha_reprogramada','Dia_reprogramado']]

In [ ]:
result_df = result_df.drop(mylist,axis=1)

In [ ]:
if repro_nuevas == 0 :
    print("Se conservó el documento")
elif repro_nuevas != 0 : 
    os.remove(current_path + "\\" + "Reprogramaciones.xlsx")
    path = current_path + "\\" + "Reprogramaciones.xlsx"   
    reprogramaciones.to_excel(path, index=False,header=True)
    with pd.ExcelWriter(path) as writer:
        reprogramaciones.to_excel(writer,index=False,sheet_name="DB")
    os.startfile(path)

In [ ]:
print(list(result_df.columns.values))
result_df = result_df[['Inicio programado', 'Orden', 'reprogramacion', 'Fecha_reprogramada',    'Plan mant.preventivo', 'Pto.tbjo.responsable',
 'Short name','Estatus', 'City', 'Status del sistema','Fe.liberación real', 'Fecha inicio real', 'Texto breve',
'Cliente', 'Denominación de objeto técnico', 'Número de serie', 'Equipo', 'Dirección', 'Semana', 'N° de Reporte ', 'ESTATUS 07 NOVIEMBRE',
'DETALLE POLITICA','Politica', 'gestora archivo anahi ', 'Frecuencia', 'Requiere Acceso', 'Tiempo aviso previo', 'Dia o fecha del Serv.',
'Horario', 'Curso', 'DC-3', 'Examenes Covid, antidoping, etc', 'Tiempo valido de prueba','Tecnicos para servicio', 'Requerimientos adicionales',
'Correo solicitud acceso', 'FACTURA FIRMADA', 'FOTOS', 'REPORTE ESPECIAL', 'Incluye materiales', 'OC','FRECUENCIA DE OC', 'PREVENTIVOS', 'CORRECTIVOS',
'Material', 'Clase de orden','Pendiente_Recoger', 'Fecha de salida', 'Comentario']]

In [ ]:
os.remove(current_path + "\\" + "Main.xlsx")
path = current_path + "\\" + "Main.xlsx"   
result_df.to_excel(path, index=False,header=True)
with pd.ExcelWriter(path) as writer:
    result_df.to_excel(writer,index=False,sheet_name="DB")
os.startfile(path)